In [1]:
import json
import csv
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import Point, MultiPoint, Polygon, MultiPolygon
from shapely.affinity import scale
import matplotlib.pyplot as plt

import glob
import os
import datetime

# WPP_Hrly_AVG_Actual_Forecasted_2016-2018
Column names vary and have to be individually treated

In [2]:
#loading all data and concatenating
path = r'/Users/margaretmccall/Documents/2020 Spring/CE 295/0 - Final Project/Data--ERCOT/WPP_Hrly_AVG_Actual_Forecasted'
all_files = glob.glob(path + "/*.csv")

dfs = []
bad_files = []

for file in all_files:
    try:
        x = pd.read_csv(file)
        dfs.append(x)
    except pd.errors.EmptyDataError:
        bad_files.append(file)

In [3]:
df_cols = []
for df in dfs:
    df_cols.append(np.array(df.columns))

In [4]:
#could have date and hour, or just hour, which presumably also contains date
just_hr_beg = []
just_hr_end = []
date_plus_hr_beg = []
date_plus_hr_end = []
just_date = []
no_match = []

for i in range(len(df_cols)): 
    if ('HOUR_BEGINNING' in df_cols[i]) & ('DELIVERY_DATE' not in df_cols[i]):
        just_hr_beg.append(i)
    elif ('HOUR_ENDING' in df_cols[i]) & ('DELIVERY_DATE' not in df_cols[i]):
        just_hr_end.append(i)
    elif ('HOUR_BEGINNING' in df_cols[i]) & ('DELIVERY_DATE' in df_cols[i]):
        date_plus_hr_beg.append(i)
    elif ('HOUR_ENDING' in df_cols[i]) & ('DELIVERY_DATE' in df_cols[i]):
        date_plus_hr_end.append(i)
    elif ('HOUR_BEGINNING' not in df_cols[i]) & ('HOUR_ENDING' not in df_cols[i]) & ('DELIVERY_DATE' in df_cols[i]):
        just_date.append(i)
    elif ('HOUR_BEGINNING' not in df_cols[i]) & ('HOUR_ENDING' not in df_cols[i]) & ('DELIVERY_DATE' not in df_cols[i]):
        no_match.append(i)

In [5]:
#covers all dataframes
lists = [just_hr_beg,
just_hr_end,
date_plus_hr_beg,
date_plus_hr_end,just_date, no_match]
for l in lists:
    print(len(l))

12061
9634
0
22080
0
0


In [6]:
#list comprehension (fast) to get list of each type of df
dfs_hrbeg = [dfs[i] for i in just_hr_beg]
dfs_hrend = [dfs[i] for i in just_hr_end]
dfs_date_hrend = [dfs[i] for i in date_plus_hr_end]

In [7]:
#concatenating all dfs of same type
dfs_hrbeg = pd.concat(dfs_hrbeg)

In [8]:
dfs_hrend = pd.concat(dfs_hrend)

In [9]:
dfs_date_hrend = pd.concat(dfs_date_hrend)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [10]:
#dropping duplicates (but why are they there?)
dfs_hrbeg.drop_duplicates(inplace=True)
dfs_hrend.drop_duplicates(inplace=True)
dfs_date_hrend.drop_duplicates(inplace=True)

In [11]:
#Converting to datetime
dfs_date_hrend['HOUR_BEGINNING'] = dfs_date_hrend['HOUR_ENDING']-1

In [12]:
dfs_date_hrend['HOUR_BEGINNING_str'] = dfs_date_hrend['HOUR_BEGINNING'].astype(str)

In [13]:
dfs_date_hrend['HOUR_BEGINNING_str'] = dfs_date_hrend['HOUR_BEGINNING_str'] + ":00"

In [17]:
dfs_date_hrend['DATE+TIME'] = dfs_date_hrend['DELIVERY_DATE'] + " " + dfs_date_hrend['HOUR_BEGINNING_str']

In [23]:
dfs_date_hrend.reset_index(inplace=True, drop=True)

In [26]:
dfs_date_hrend['DT'] = pd.to_datetime(dfs_date_hrend['DATE+TIME'])

In [29]:
import pickle
filename = 'dfs_date_hrend.pickle'
with open(filename, 'wb') as fp:
    pickle.dump(dfs_date_hrend, fp)

In [30]:
dfs_hrbeg['HOUR_BEGINNING'] = pd.to_datetime(dfs_hrbeg['HOUR_BEGINNING'])

In [31]:
dfs_hrend['HOUR_ENDING'] = pd.to_datetime(dfs_hrend['HOUR_ENDING'])

In [32]:
filename = 'dfs_hrbeg.pickle'
with open(filename, 'wb') as fp:
    pickle.dump(dfs_hrbeg, fp)
filename = 'dfs_hrend.pickle'
with open(filename, 'wb') as fp:
    pickle.dump(dfs_hrend, fp)

In [62]:
filename = 'dfs_date_hrend.pickle' #loading again; screwed up
with open(filename, 'rb') as fp:
    dfs_date_hrend = pickle.load(fp)

In [34]:
dfs_hrend['HOUR_BEGINNING'] = dfs_hrend['HOUR_ENDING'] - datetime.timedelta(hours=1)
dfs_hrend.drop(columns='HOUR_ENDING',inplace=True)

In [63]:
#aligning columns
dfs_date_hrend['HOUR_BEGINNING'] = dfs_date_hrend['DT']
dfs_date_hrend.drop(columns=['HOUR_BEGINNING_str','DATE+TIME','DT','DELIVERY_DATE'],
                   inplace=True)

In [64]:
set(dfs_hrbeg.columns) - set(dfs_hrend.columns)
set(dfs_date_hrend.columns) - set(dfs_hrend.columns)

{'ACTUAL_LZ_NORTH',
 'ACTUAL_LZ_SOUTH_HOUSTON',
 'ACTUAL_LZ_WEST',
 'COP_HSL_LZ_NORTH',
 'COP_HSL_LZ_SOUTH_HOUSTON',
 'COP_HSL_LZ_WEST',
 'HOUR_ENDING',
 'STWPF_LZ_NORTH',
 'STWPF_LZ_SOUTH_HOUSTON',
 'STWPF_LZ_WEST',
 'WGRPP_LZ_NORTH',
 'WGRPP_LZ_SOUTH_HOUSTON',
 'WGRPP_LZ_WEST'}

In [65]:
dfs_date_hrend.drop(columns=list(set(dfs_date_hrend.columns) - set(dfs_hrend.columns)),inplace=True)

In [66]:
#sorting
dfs_hrbeg.sort_values(by=['HOUR_BEGINNING'], inplace=True)
dfs_hrend.sort_values(by=['HOUR_BEGINNING'], inplace=True)
dfs_date_hrend.sort_values(by=['HOUR_BEGINNING'], inplace=True)

In [69]:
merged_wind = pd.concat([dfs_hrbeg, dfs_hrend, dfs_date_hrend])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [71]:
merged_wind.drop_duplicates(inplace=True)

In [ ]:
merged_wind.sort_values(by=['HOUR_BEGINNING'], inplace=True)
merged_wind.drop(columns=['COP_HSL_SOUTH_HOUSTON','COP_HSL_SYSTEM_WIDE','COP_HSL_WEST_NORTH',
                         'DSTFlag','WGRPP_SOUTH_HOUSTON','WGRPP_SYSTEM_WIDE','WGRPP_WEST_NORTH'], inplace=True)
merged_wind.head()

In [89]:
merged_wind.reset_index(inplace=True, drop=True)

In [76]:
filename = 'merged_wind.pickle'
with open(filename, 'wb') as fp:
    pickle.dump(merged_wind, fp)

In [90]:
merged_wind['year'] = merged_wind['HOUR_BEGINNING'].dt.year

In [91]:
merged_wind['hour'] = merged_wind['HOUR_BEGINNING'].dt.hour

In [92]:
merged_wind['date'] = merged_wind['HOUR_BEGINNING'].dt.date

In [100]:
#for some reason there are millions of extra entries--seemsl ike some of this was sub-hourly.
#averaging
grouped_wind = merged_wind.groupby('HOUR_BEGINNING').mean()
grouped_wind.drop(columns=['year','hour'], inplace=True)

In [102]:
grouped_wind.reset_index(inplace=True)

In [104]:
grouped_wind['date'] = grouped_wind['HOUR_BEGINNING'].dt.date
grouped_wind['hr_beg'] = grouped_wind['HOUR_BEGINNING'].dt.hour

In [106]:
grouped_wind[grouped_wind['date']==datetime.date(2014,3,9)]

,HOUR_BEGINNING,ACTUAL_SOUTH_HOUSTON,ACTUAL_SYSTEM_WIDE,ACTUAL_WEST_NORTH,STWPF_SOUTH_HOUSTON,STWPF_SYSTEM_WIDE,STWPF_WEST_NORTH,date,hr_beg
1656,2014-03-09 00:00:00,2104.29,6492.645,4388.355,1491.036000,4308.986000,2817.950000,2014-03-09,0
1657,2014-03-09 02:00:00,1737.14,4077.220,2340.080,1174.483333,4300.027083,3125.543750,2014-03-09,2
1658,2014-03-09 03:00:00,1998.84,3697.000,1698.160,1707.131250,4086.968750,2379.837500,2014-03-09,3
1659,2014-03-09 04:00:00,1871.61,2720.460,848.850,1719.777083,3747.087500,2027.310417,2014-03-09,4
1660,2014-03-09 05:00:00,1717.74,2279.310,561.570,1684.489583,3367.814583,1683.325000,2014-03-09,5
1661,2014-03-09 06:00:00,1336.50,1835.840,499.340,1619.310417,3150.118750,1530.808333,2014-03-09,6
1662,2014-03-09 07:00:00,1202.32,1805.330,603.010,1555.152083,3080.647917,1525.495833,2014-03-09,7
1663,2014-03-09 08:00:00,1336.05,2033.380,697.330,1445.647917,2803.164583,1357.516667,2014-03-09,8
1664,2014-03-09 09:00:00,1167.25,1577.710,410.460,1380.006250,2641.035417,1261.029167,2014-03-09,9
1665,2014-03-09 10:00:00,1219.14,1533.000,313.860,1280.100000,2508.204167,1228.104167,2014-03-09,10


In [107]:
#TIME ZONES / DST
import pytz
from pytz import timezone

In [108]:
def get_utc(df):
    """Requires dataframe with column 'dt' with datetime"""
    central = timezone('America/Chicago')
    df['Central'] = df['dt'].apply(lambda x: central.localize(x))
    df['UTC'] = df['Central'].apply(lambda x: pytz.utc.normalize(x))
    return df

In [111]:
grouped_wind.rename(columns={'HOUR_BEGINNING':'dt'}, inplace=True)

In [112]:
grouped_wind = get_utc(grouped_wind)

In [113]:
grouped_wind['date'] = grouped_wind['UTC'].dt.date
grouped_wind['hr_beg'] = grouped_wind['UTC'].dt.hour

In [115]:
grouped_wind.drop(columns=['Central','UTC','dt'], inplace=True)

In [118]:
#dealing with NANs
#ok, only keeping actual_system_wide, I guess...
grouped_wind = grouped_wind[['date','hr_beg','ACTUAL_SYSTEM_WIDE','STWPF_SYSTEM_WIDE']]

In [127]:
grouped_wind = grouped_wind[grouped_wind['date']<datetime.date(2019,1,1)]

In [129]:
grouped_wind.fillna(method='bfill',axis=0, inplace=True)

In [132]:
grouped_wind.to_csv('wind.csv', index=False)

Seems like we actually care about actual and STWPF (short-term wind power forecast), less WGRPP (wind gen resource power potential) or COP HSL (current operating plan high sustainable limit) http://www.ercot.com/glossary/h